# Operations

`DataFrame` and `Series` objects allow for several operations to take place either on a single object, or between two or more objects.

For this section, we will import a complementary dataset of transits (voyages) of each vessel. This is a much larger dataset, since ships each have multiple transits over several years.

For example, we can perform arithmetic on the elements of two objects, such as combining distances travelled across years. First, let's artificially construct two Series, consisting of segment lengths travelled in 2009 and 2010:

In [49]:
segments = pd.read_csv('../data/AIS/transit_segments.csv', parse_dates=['st_time'])

In [51]:
segments.shape

(262526, 11)

Extract the year from the date (more on dates/times later!)

In [54]:
segments['year'] = segments.st_time.dt.year
segments.year.head()

0    2009
1    2009
2    2009
3    2009
4    2009
Name: year, dtype: int64

In [55]:
segments2009 = segments[segments.year==2009].copy()
lengths2009 = pd.Series({name: segments2009[segments2009.name==name].seg_length.sum() 
                         for name in segments2009.name.unique()})

In [56]:
segments2010 = segments[segments.year==2010].copy()
lengths2010 = pd.Series({name: segments2010[segments2010.name==name].seg_length.sum() 
                         for name in segments2010.name.unique()})

We will learn a much more efficient way of performing this operation when we get to `GroupBy` operations a little later on.

In [59]:
length2009 = segments.loc[segments.year==2009, 'seg_length']
length2009.index = segments.name[segments.year==2009]

length2010 = segments.loc[segments.year==2010, 'seg_length']
length2010.index = segments.name[segments.year==2010]

Now, let's add them together, in hopes of getting 2-year travel totals:

In [61]:
seg_lengths = lengths2009 + lengths2010
seg_lengths

$ Vega Azurit $            NaN
+ Alida Gorthon +          NaN
+alida Gorthon+           63.0
+margit Gorthon+           NaN
0mega Theodore             NaN
1700 Somewhere             NaN
1st Lt Alex Bonnyman       NaN
1st Lt Jack Lummus         NaN
205551000                  NaN
210235000                  NaN
210670000                 43.5
212043000                  5.2
212412000                  NaN
235067192                  NaN
235075024                  NaN
240716000                  NaN
245554000                  NaN
247102700                  NaN
247242400                  NaN
247281100                  NaN
247645000                 55.4
257356000                  NaN
304716000                  NaN
305624000                  NaN
311003300                  7.7
311003400                  NaN
311011100                  NaN
311011400                  NaN
311022500                  NaN
319015800                  NaN
                         ...  
Zim California          1166.4
Zim Haif

Pandas' data alignment places `NaN` values for labels that do not overlap in the two Series. In fact, the majority of ships only have data for one of the two years.

In [62]:
seg_lengths.notnull().mean()

0.35575589459084606

While we do want the operation to honor the data labels in this way, we probably do not want the missing values to be filled with `NaN`. We can use the `add` method to calculate ship travel totals by using the `fill_value` argument to insert a zero for years that do not have data for a particular vessel:

In [63]:
lengths2009.add(lengths2010, fill_value=0)

$ Vega Azurit $           69.0
+ Alida Gorthon +        497.2
+alida Gorthon+           63.0
+margit Gorthon+         342.3
0mega Theodore            62.6
1700 Somewhere             7.6
1st Lt Alex Bonnyman      96.4
1st Lt Jack Lummus         1.3
205551000                  7.2
210235000                  1.2
210670000                 43.5
212043000                  5.2
212412000                  4.9
235067192                 19.5
235075024                  1.1
240716000                 72.9
245554000                  4.1
247102700                 24.1
247242400                  8.9
247281100                 21.1
247645000                 55.4
257356000                135.3
304716000                  4.0
305624000                 53.6
311003300                  7.7
311003400                  4.3
311011100                 18.9
311011400                  3.3
311022500                 19.6
319015800                 10.7
                         ...  
Zim California          1166.4
Zim Haif

Operations can also be **broadcast** between rows or columns.

For example, if we subtract the maximum LOA (ship length) from the `max_loa` column, we get a the differences between the size of a given vessel to the largest vessel.

In [64]:
vessels.max_loa - vessels.max_loa.max()

mmsi
1           -656.0
9           -750.0
21          -604.0
74          -657.0
103         -657.0
310         -765.0
3011        -652.0
4731        -782.0
15151       -637.0
46809       -790.0
80404       -783.0
82003       -783.0
298716      -777.0
366235      -605.0
439541      -757.0
453556      -604.0
505843      -792.0
527918      -792.0
565026      -780.0
572329      -772.0
587370      -779.0
641114      -792.0
642262      -757.0
693559      -772.0
883085      -607.0
1193046     -143.0
1193946     -745.0
1233916     -792.0
1239468     -790.0
3041300     -694.0
             ...  
720728000   -714.0
720754000   -780.0
720768000   -783.0
725004700   -652.0
725005560   -635.0
725005570   -637.0
725011300   -645.0
725018300   -636.0
725019006   -699.0
725021000   -624.0
725022000   -595.0
730000161   -708.0
730010001   -736.0
730026000   -693.0
730031000   -692.0
735057548   -584.0
735059037   -734.0
760101000   -664.0
770576100   -752.0
812719000   -760.0
857632392   -792.0
8669468

Or, looking at things row-wise, we can see how a particular vessel compares with the rest of the group with respect to important features:

In [65]:
vessels[vessels.max_loa==vessels.max_loa.max()]

,num_names,names,sov,flag,flag_type,num_loas,loa,max_loa,num_types,type
mmsi,,,,,,,,,,
354092000,1,Msc Laura,N,Panama (Republic of),Foreign,3,300.0/301.0/812.0,812.0,1,Cargo


In [66]:
recs = vessels[['num_names','num_loas', 'max_loa', 'num_types']]
diff = recs - recs.loc[354092000]
diff[:10]

,num_names,num_loas,max_loa,num_types
mmsi,,,,
1,7.0,4.0,-656.0,3.0
9,2.0,-1.0,-750.0,1.0
21,0.0,-2.0,-604.0,0.0
74,1.0,-2.0,-657.0,0.0
103,2.0,-1.0,-657.0,1.0
310,0.0,-2.0,-765.0,0.0
3011,0.0,-2.0,-652.0,0.0
4731,0.0,-2.0,-782.0,0.0
15151,1.0,-1.0,-637.0,0.0


We can also apply functions to each column or row of a `DataFrame`

In [70]:
recs.apply(np.median)

num_names      1.0
num_loas       1.0
max_loa      183.0
num_types      1.0
dtype: float64

In [71]:
def range_calc(x):
    return x.max() - x.min()

In [72]:
recs.apply(range_calc)

num_names    167.0
num_loas      83.0
max_loa      812.0
num_types     13.0
dtype: float64

## Sorting and Ranking

Pandas objects include methods for re-ordering data.

In [73]:
segments.sort_index().head()

,mmsi,name,transit,segment,seg_length,avg_sog,min_sog,max_sog,pdgt10,st_time,end_time,year
0,1,Us Govt Ves,1,1,5.1,13.2,9.2,14.5,96.5,2009-02-10 16:03:00,2/10/09 16:27,2009
1,1,Dredge Capt Frank,1,1,13.5,18.6,10.4,20.6,100.0,2009-04-06 14:31:00,4/6/09 15:20,2009
2,1,Us Gov Vessel,1,1,4.3,16.2,10.3,20.5,100.0,2009-04-06 14:36:00,4/6/09 14:55,2009
3,1,Us Gov Vessel,2,1,9.2,15.4,14.5,16.1,100.0,2009-04-10 17:58:00,4/10/09 18:34,2009
4,1,Dredge Capt Frank,2,1,9.2,15.4,14.6,16.2,100.0,2009-04-10 17:59:00,4/10/09 18:35,2009


In [74]:
segments.sort_index(ascending=False).head()

,mmsi,name,transit,segment,seg_length,avg_sog,min_sog,max_sog,pdgt10,st_time,end_time,year
262525,999999999,Triple Attraction,8,1,19.8,18.6,16.1,19.5,100.0,2010-06-18 10:19:00,6/18/10 11:22,2010
262524,999999999,Triple Attraction,7,1,31.5,14.2,13.4,15.1,100.0,2010-06-18 02:52:00,6/18/10 5:03,2010
262523,999999999,Triple Attraction,6,1,17.4,17.0,14.7,18.4,100.0,2010-06-17 19:16:00,6/17/10 20:17,2010
262522,999999999,Triple Attraction,4,1,18.7,19.2,18.4,19.9,100.0,2010-06-15 21:32:00,6/15/10 22:29,2010
262521,999999999,Triple Attraction,3,1,5.3,20.0,19.6,20.4,100.0,2010-06-15 12:49:00,6/15/10 13:05,2010


Try sorting the **columns** instead of the rows, in ascending order:

In [75]:
segments.sort_index(axis=1).head()

,avg_sog,end_time,max_sog,min_sog,mmsi,name,pdgt10,seg_length,segment,st_time,transit,year
0,13.2,2/10/09 16:27,14.5,9.2,1,Us Govt Ves,96.5,5.1,1,2009-02-10 16:03:00,1,2009
1,18.6,4/6/09 15:20,20.6,10.4,1,Dredge Capt Frank,100.0,13.5,1,2009-04-06 14:31:00,1,2009
2,16.2,4/6/09 14:55,20.5,10.3,1,Us Gov Vessel,100.0,4.3,1,2009-04-06 14:36:00,1,2009
3,15.4,4/10/09 18:34,16.1,14.5,1,Us Gov Vessel,100.0,9.2,1,2009-04-10 17:58:00,2,2009
4,15.4,4/10/09 18:35,16.2,14.6,1,Dredge Capt Frank,100.0,9.2,1,2009-04-10 17:59:00,2,2009


We can also use `sort_values` to sort a `Series` by value, rather than by label.

In [76]:
segments.seg_length.sort_values(ascending=False).head(10)

181047    1882.0
181108    1691.0
149218    1675.3
49709     1647.8
149236    1644.2
50604     1557.0
106374    1517.3
140513    1502.4
140555    1491.6
50665     1489.2
Name: seg_length, dtype: float64

For a `DataFrame`, we can sort according to the values of one or more columns using the `by` argument of `sort_values`:

In [77]:
segments[['avg_sog','max_sog','min_sog']].sort_values(ascending=[False,True], 
                                           by=['max_sog', 'min_sog']).head(10)

,avg_sog,max_sog,min_sog
82222,1.5,49.9,0.0
167724,22.3,49.9,0.0
215785,2.8,49.9,0.0
119381,6.6,49.9,0.1
12479,12.2,49.9,3.5
119476,1.9,49.8,0.0
160406,9.9,49.8,0.0
162241,1.9,49.8,0.0
146139,9.6,49.8,0.1
160395,10.1,49.8,0.1


A related method `rank` does not re-arrange data, but instead returns an index that ranks each value relative to others in the Series.

In [78]:
segments.avg_sog.rank()

0         204327.5
1         255825.5
2         243231.5
3         235514.0
4         235514.0
5         262505.0
6         262506.5
7          41350.5
8          77624.0
9          65892.0
10          1122.5
11         63776.5
12         45795.5
13         86648.5
14          2122.5
15         13314.0
16         37209.5
17        212583.0
18        112262.5
19        199124.0
20        245654.5
21        127636.5
22        143948.5
23        207656.0
24        239600.5
25         65892.0
26        136465.0
27         90087.0
28        195562.5
29        140367.5
            ...   
262496     23629.0
262497     23629.0
262498     24320.5
262499     25795.0
262500     31198.5
262501     33486.5
262502     25039.5
262503     30133.0
262504     34668.0
262505     19816.5
262506     18617.0
262507     26591.0
262508     27400.0
262509     20425.0
262510     35900.0
262511     12804.5
262512     25039.5
262513     15288.5
262514     18039.5
262515     28256.0
262516     25795.0
262517     1

Ties are assigned the mean value of the tied ranks, which may result in decimal values.

In [80]:
pd.Series([100,100,100]).rank()

0    2.0
1    2.0
2    2.0
dtype: float64

Alternatively, you can break ties via one of several methods, such as by the order in which they occur in the dataset:

In [81]:
segments.avg_sog.rank(method='first').sort_values()

663            1.0
664            2.0
665            3.0
666            4.0
754            5.0
756            6.0
788            7.0
795            8.0
800            9.0
805           10.0
807           11.0
808           12.0
814           13.0
817           14.0
818           15.0
1113          16.0
1151          17.0
1152          18.0
1156          19.0
1186          20.0
1195          21.0
1227          22.0
1270          23.0
1358          24.0
2434          25.0
2436          26.0
2437          27.0
2440          28.0
2441          29.0
2443          30.0
            ...   
133971    262497.0
193181    262498.0
179881    262499.0
179882    262500.0
73207     262501.0
73225     262502.0
75990     262503.0
133963    262504.0
5         262505.0
6         262506.0
165993    262507.0
75731     262508.0
73223     262509.0
75562     262510.0
73216     262511.0
73224     262512.0
1425      262513.0
74280     262514.0
178394    262515.0
179152    262516.0
73226     262517.0
75989     26

Calling the `DataFrame`'s `rank` method results in the ranks of all columns:

In [82]:
segments.rank(ascending=False).head()

,mmsi,name,transit,segment,seg_length,avg_sog,min_sog,max_sog,pdgt10,st_time,end_time,year
0,262522.5,14015.0,256508.0,142468.5,254075.0,58199.5,62367.5,92016.5,55855.0,245583.0,176864.5,220840.5
1,262522.5,194589.5,256508.0,142468.5,237380.5,6701.5,32270.5,13177.0,19842.0,236699.5,119561.0,220840.5
2,262522.5,14047.5,256508.0,142468.5,255526.5,19295.5,33803.5,13784.5,19842.0,236696.0,119562.0,220840.5
3,262522.5,14047.5,244908.5,142468.5,247002.0,27013.0,7614.0,61125.0,19842.0,236013.0,137215.0,220840.5
4,262522.5,194589.5,244908.5,142468.5,247002.0,27013.0,7372.0,59654.0,19842.0,236012.0,137214.0,220840.5


## Hierarchical indexing

While the `mmsi` (Maritime Mobile Service Identity) field represents a unique identifier in the `vessels` table, it does not in the `segments` table.

We can use hierarchical indexing to build a **unique index** that may be more meaningful than the ordered set of integers that are given as a default index.

In [87]:
segments_hier = segments.set_index(['mmsi', 'name', 'year', 'transit', 'segment']).sort_index(ascending=True)
segments_hier.head()

seg_length  avg_sog  min_sog  \
mmsi name              year transit segment                                 
1    Bil Holman Dredge 2010 1       1              17.4     34.3     33.9   
                                    2              76.0     34.4     33.8   
     Dredge Capt Frank 2009 1       1              13.5     18.6     10.4   
                            2       1               9.2     15.4     14.6   
     S.d. Gumel        2011 1       1              13.7      6.9      3.0   

                                             max_sog  pdgt10  \
mmsi name              year transit segment                    
1    Bil Holman Dredge 2010 1       1           35.0   100.0   
                                    2           34.9   100.0   
     Dredge Capt Frank 2009 1       1           20.6   100.0   
                            2       1           16.2   100.0   
     S.d. Gumel        2011 1       1           14.0    38.2   

                                                        st_time       end_time  
mmsi name              year transit segment                                     
1    Bil Holman Dredge 2010 1       1       2010-03-20 16:06:00  3/20/10 16:36  
                                    2       2010-03-20 18:05:00  3/20/10 18:48  
     Dredge Capt Frank 2009 1       1       2009-04-06 14:31:00   4/6/09 15:20  
                            2       1       2009-04-10 17:59:00  4/10/09 18:35  
     S.d. Gumel        2011 1       1       2011-05-04 11:28:00   5/4/11 13:30

This index is a `MultiIndex` object that consists of a sequence of tuples, the elements of which is some combination of the three columns used to create the index. Where there are multiple repeated values, Pandas does not print the repeats, making it easy to identify groups of values.

In [88]:
segments_hier.index.is_unique

True

Try using this hierarchical index to retrieve the first segment of the tenth transit of the *Sentinel* (mmsi=366766980):

In [89]:
segments_hier.loc[(366766980, 'Sentinel', 2009, 10, 1)]

seg_length                   45.5
avg_sog                       9.8
min_sog                         6
max_sog                      10.4
pdgt10                       37.5
st_time       2009-01-23 07:08:00
end_time            1/23/09 10:49
Name: (366766980, Sentinel, 2009, 10, 1), dtype: object

With a hierachical index, we can select subsets of the data based on a *partial* index:

In [91]:
segments_hier.loc[9]

seg_length  avg_sog  min_sog  max_sog  \
name       year transit segment                                          
Shearwater 2010 4       1              11.6      8.8      7.9     10.3   
                8       1              11.6      8.3      7.4      9.2   
                9       1               7.3      0.5      0.0      9.3   
                12      1              17.8      8.2      7.3      8.7   
                14      1              12.7      7.2      6.4      7.8   
                15      1              19.3      9.1      6.5     10.2   
                16      1              31.4      0.8      0.0      9.4   
                17      1              47.2      3.4      0.0     11.8   

                                 pdgt10             st_time       end_time  
name       year transit segment                                             
Shearwater 2010 4       1           0.0 2010-06-05 11:23:00   6/5/10 12:44  
                8       1           0.0 2010-06-08 11:03:00   6/8/10 12:29  
                9       1           0.0 2010-06-14 17:07:00  6/15/10 14:45  
                12      1           0.0 2010-06-17 16:35:00  6/17/10 18:43  
                14      1           0.0 2010-07-11 11:51:00  7/11/10 13:35  
                15      1           5.4 2010-07-12 15:33:00  7/12/10 17:36  
                16      1           0.0 2010-07-20 19:23:00  7/23/10 15:32  
                17      1          25.1 2010-09-10 14:00:00  9/11/10 14:42

Hierarchical indices can be created on either or both axes. Here is a trivial example:

In [92]:
frame = pd.DataFrame(np.arange(12).reshape(( 4, 3)), 
                  index =[['a', 'a', 'b', 'b'], [1, 2, 1, 2]], 
                  columns =[['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])

frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

If you want to get fancy, both the row and column indices themselves can be given names:

In [93]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

With this, we can do all sorts of custom indexing:

In [94]:
frame.loc['a', 'Ohio']

color,Green,Red
key2,,
1,0,1
2,3,4


#### Exercise: Try retrieving the value corresponding to `b2` in `Colorado`:

In [103]:
frame.index

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['key1', 'key2'])

In [102]:
frame.loc[('b', 2), 'Colorado']

color
Green    11
Name: (b, 2), dtype: int64

Additionally, the order of the set of indices in a hierarchical `MultiIndex` can be changed by swapping them pairwise:

In [104]:
segments_hier.swaplevel('mmsi', 'name').head()

seg_length  avg_sog  min_sog  \
name              mmsi year transit segment                                 
Bil Holman Dredge 1    2010 1       1              17.4     34.3     33.9   
                                    2              76.0     34.4     33.8   
Dredge Capt Frank 1    2009 1       1              13.5     18.6     10.4   
                            2       1               9.2     15.4     14.6   
S.d. Gumel        1    2011 1       1              13.7      6.9      3.0   

                                             max_sog  pdgt10  \
name              mmsi year transit segment                    
Bil Holman Dredge 1    2010 1       1           35.0   100.0   
                                    2           34.9   100.0   
Dredge Capt Frank 1    2009 1       1           20.6   100.0   
                            2       1           16.2   100.0   
S.d. Gumel        1    2011 1       1           14.0    38.2   

                                                        st_time       end_time  
name              mmsi year transit segment                                     
Bil Holman Dredge 1    2010 1       1       2010-03-20 16:06:00  3/20/10 16:36  
                                    2       2010-03-20 18:05:00  3/20/10 18:48  
Dredge Capt Frank 1    2009 1       1       2009-04-06 14:31:00   4/6/09 15:20  
                            2       1       2009-04-10 17:59:00  4/10/09 18:35  
S.d. Gumel        1    2011 1       1       2011-05-04 11:28:00   5/4/11 13:30

In [105]:
segments_hier.swaplevel('mmsi', 'name').loc['Sentinel'].head()

seg_length  avg_sog  min_sog  max_sog  pdgt10  \
mmsi      year transit segment                                                  
366766980 2008 1       1              44.6      6.6      4.9      9.3     0.0   
               2       1              33.6      8.2      4.6      8.9     0.0   
               3       1              27.0      6.7      2.8     11.3    10.0   
               4       1              35.4     10.0      6.7     10.4    69.1   
               5       1             100.9      5.2      2.6      8.9     0.0   

                                           st_time        end_time  
mmsi      year transit segment                                      
366766980 2008 1       1       2008-11-10 05:00:00  11/10/08 11:49  
               2       1       2008-11-12 09:53:00  11/12/08 14:00  
               3       1       2008-11-24 07:02:00  11/24/08 11:01  
               4       1       2008-12-05 20:20:00   12/5/08 23:53  
               5       1       2008-12-17 00:52:00  12/17/08 20:18

Alternately, one can index based on an arbitrary level using the `xs` (cross-section) method:

In [107]:
segments_hier.xs(4, level='transit').head()

seg_length  avg_sog  min_sog  max_sog  \
mmsi name          year segment                                          
9    Shearwater    2010 1              11.6      8.8      7.9     10.3   
21   Us Gov Vessel 2011 1              18.0      9.7      4.6     15.2   
103  Ron G         2009 1              62.6      8.9      6.8      9.8   
310  Arabella      2009 1              19.6      9.7      9.3     10.3   
                        2              28.8      9.3      8.6     11.5   

                                 pdgt10             st_time       end_time  
mmsi name          year segment                                             
9    Shearwater    2010 1           0.0 2010-06-05 11:23:00   6/5/10 12:44  
21   Us Gov Vessel 2011 1          76.3 2011-04-25 16:37:00  4/25/11 18:25  
103  Ron G         2009 1           0.0 2009-12-20 17:57:00  12/21/09 0:59  
310  Arabella      2009 1           2.8 2009-07-23 02:47:00   7/23/09 4:45  
                        2           2.6 2009-07-23 04:46:00   7/23/09 7:47

Perhaps the easiest way to extract data from arbitrary levels of a `MultiIndex` is to use the `IndexSlice` function. For example, if we want the first two transits from all trips in 2009 and 2010:

In [109]:
segments_hier.loc[pd.IndexSlice[:, :, 2009:2010, [1,2]], 'seg_length'].head(12)

mmsi  name               year  transit  segment
1     Bil Holman Dredge  2010  1        1          17.4
                                        2          76.0
      Dredge Capt Frank  2009  1        1          13.5
                               2        1           9.2
      Us Gov Vessel      2009  1        1           4.3
                               2        1           9.2
      Us Govt Ves        2009  1        1           5.1
74    Mcfaul             2009  1        2           1.4
      Sarah Bell         2009  1        1           7.4
103   Ron G              2009  1        1          87.5
                                        3           1.4
                               2        1          30.7
Name: seg_length, dtype: float64

The `:` tells pandas to include the entire level. What `IndexSlice` does is generate a Python `slice` object that represents what is required from each level. So, the above is equivalent to:

    (slice(None, None, None), slice(None, None, None), [2009, 2010], [1, 2])

Data can also be sorted by any index level, using `sort_index` with the `level` argument appropriately specified:

In [114]:
segments_hier.sort_index(level='transit', ascending=True).head()

seg_length  avg_sog  min_sog  \
mmsi name              year transit segment                                 
1    Bil Holman Dredge 2010 1       1              17.4     34.3     33.9   
                                    2              76.0     34.4     33.8   
     Dredge Capt Frank 2009 1       1              13.5     18.6     10.4   
     S.d. Gumel        2011 1       1              13.7      6.9      3.0   
     Us Gov Vessel     2009 1       1               4.3     16.2     10.3   

                                             max_sog  pdgt10  \
mmsi name              year transit segment                    
1    Bil Holman Dredge 2010 1       1           35.0   100.0   
                                    2           34.9   100.0   
     Dredge Capt Frank 2009 1       1           20.6   100.0   
     S.d. Gumel        2011 1       1           14.0    38.2   
     Us Gov Vessel     2009 1       1           20.5   100.0   

                                                        st_time       end_time  
mmsi name              year transit segment                                     
1    Bil Holman Dredge 2010 1       1       2010-03-20 16:06:00  3/20/10 16:36  
                                    2       2010-03-20 18:05:00  3/20/10 18:48  
     Dredge Capt Frank 2009 1       1       2009-04-06 14:31:00   4/6/09 15:20  
     S.d. Gumel        2011 1       1       2011-05-04 11:28:00   5/4/11 13:30  
     Us Gov Vessel     2009 1       1       2009-04-06 14:36:00   4/6/09 14:55

## Missing data

The occurence of missing data is so prevalent that it pays to use tools like Pandas, which seamlessly integrates missing data handling so that it can be dealt with easily, and in the manner required by the analysis at hand.

Missing data are represented in `Series` and `DataFrame` objects by the `NaN` floating point value. However, `None` is also treated as missing, since it is commonly used as such in other contexts (*e.g.* NumPy).

In [117]:
foo = pd.Series([np.nan, -3, None, 'foobar', np.inf])
foo

0       NaN
1        -3
2      None
3    foobar
4       inf
dtype: object

In [118]:
foo.isnull()

0     True
1    False
2     True
3    False
4    False
dtype: bool

To illustrate how to deal with missing values in pandas, we will import an educational database, which reports test scores for children with hearing disabilities, along with associated covariates, several of which are missing.

In [119]:
testing = pd.read_csv('../data/test_scores.csv', index_col=0)
testing.head(10)

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
0,40,0,2.0,2.0,False,NaN,55,1.0,NaN,False,False
1,31,1,0.0,NaN,False,0.0,53,0.0,0.0,False,False
2,83,1,1.0,1.0,True,0.0,52,1.0,NaN,False,True
3,75,0,3.0,NaN,False,0.0,55,0.0,1.0,False,False
5,62,0,0.0,4.0,False,1.0,50,0.0,NaN,False,False
6,85,1,1.0,1.0,False,0.0,53,0.0,NaN,False,False
7,39,0,2.0,3.0,True,1.0,50,1.0,1.0,False,True
9,98,0,0.0,1.0,False,NaN,48,0.0,0.0,True,False
10,64,0,0.0,0.0,True,0.0,54,0.0,NaN,False,True
12,109,1,1.0,0.0,False,0.0,48,1.0,1.0,True,False


Missing values may be dropped or indexed out:

By default, `dropna` drops entire rows in which one or more values are missing.

In [120]:
testing.dropna().head(10)

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
7,39,0,2.0,3.0,True,1.0,50,1.0,1.0,False,True
12,109,1,1.0,0.0,False,0.0,48,1.0,1.0,True,False
14,91,0,2.0,0.0,False,0.0,58,1.0,1.0,True,True
19,65,1,0.0,2.0,False,0.0,50,1.0,1.0,False,True
21,71,1,1.0,2.0,False,1.0,58,0.0,0.0,True,True
23,82,0,1.0,0.0,False,1.0,59,0.0,1.0,True,False
25,97,0,1.0,0.0,False,1.0,56,1.0,1.0,False,False
26,95,1,1.0,0.0,True,0.0,55,0.0,1.0,True,False
27,98,1,1.0,0.0,False,0.0,52,0.0,1.0,True,False
32,115,0,1.0,0.0,False,0.0,55,0.0,1.0,False,False


A similar result can be obtained via indexing.

In [121]:
testing.isnull().head()

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
0,False,False,False,False,False,True,False,False,True,False,False
1,False,False,False,True,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,True,False,False
3,False,False,False,True,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,True,False,False


In [122]:
testing[testing.notnull()].head()

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
0,40,0,2.0,2.0,False,NaN,55,1.0,NaN,False,False
1,31,1,0.0,NaN,False,0.0,53,0.0,0.0,False,False
2,83,1,1.0,1.0,True,0.0,52,1.0,NaN,False,True
3,75,0,3.0,NaN,False,0.0,55,0.0,1.0,False,False
5,62,0,0.0,4.0,False,1.0,50,0.0,NaN,False,False


This can be overridden by passing the `how='all'` argument, which only drops a row when every field is a missing value.

In [123]:
testing.dropna(how='all')

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
0,40,0,2.0,2.0,False,NaN,55,1.0,NaN,False,False
1,31,1,0.0,NaN,False,0.0,53,0.0,0.0,False,False
2,83,1,1.0,1.0,True,0.0,52,1.0,NaN,False,True
3,75,0,3.0,NaN,False,0.0,55,0.0,1.0,False,False
5,62,0,0.0,4.0,False,1.0,50,0.0,NaN,False,False
6,85,1,1.0,1.0,False,0.0,53,0.0,NaN,False,False
7,39,0,2.0,3.0,True,1.0,50,1.0,1.0,False,True
9,98,0,0.0,1.0,False,NaN,48,0.0,0.0,True,False
10,64,0,0.0,0.0,True,0.0,54,0.0,NaN,False,True
12,109,1,1.0,0.0,False,0.0,48,1.0,1.0,True,False


This can be customized further by specifying how many values need to be present before a row is dropped via the `thresh` argument.

In [124]:
testing.dropna(thresh=10)

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
1,31,1,0.0,NaN,False,0.0,53,0.0,0.0,False,False
2,83,1,1.0,1.0,True,0.0,52,1.0,NaN,False,True
3,75,0,3.0,NaN,False,0.0,55,0.0,1.0,False,False
5,62,0,0.0,4.0,False,1.0,50,0.0,NaN,False,False
6,85,1,1.0,1.0,False,0.0,53,0.0,NaN,False,False
7,39,0,2.0,3.0,True,1.0,50,1.0,1.0,False,True
9,98,0,0.0,1.0,False,NaN,48,0.0,0.0,True,False
10,64,0,0.0,0.0,True,0.0,54,0.0,NaN,False,True
12,109,1,1.0,0.0,False,0.0,48,1.0,1.0,True,False
13,111,0,0.0,0.0,False,1.0,48,1.0,NaN,False,True


This is typically used in time series applications, where there are repeated measurements that are incomplete for some subjects.

### Exercise

Try using the `axis` argument to drop columns with missing values:

In [ ]:
# Write your answer here

Rather than omitting missing data from an analysis, in some cases it may be suitable to fill the missing value in, either with a default value (such as zero) or a value that is either imputed or carried forward/backward from similar data points. We can do this programmatically in Pandas with the `fillna` argument.

In [125]:
testing.fillna(0).head(10)

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
0,40,0,2.0,2.0,False,0.0,55,1.0,0.0,False,False
1,31,1,0.0,0.0,False,0.0,53,0.0,0.0,False,False
2,83,1,1.0,1.0,True,0.0,52,1.0,0.0,False,True
3,75,0,3.0,0.0,False,0.0,55,0.0,1.0,False,False
5,62,0,0.0,4.0,False,1.0,50,0.0,0.0,False,False
6,85,1,1.0,1.0,False,0.0,53,0.0,0.0,False,False
7,39,0,2.0,3.0,True,1.0,50,1.0,1.0,False,True
9,98,0,0.0,1.0,False,0.0,48,0.0,0.0,True,False
10,64,0,0.0,0.0,True,0.0,54,0.0,0.0,False,True
12,109,1,1.0,0.0,False,0.0,48,1.0,1.0,True,False


In [127]:
testing

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
0,40,0,2.0,2.0,False,NaN,55,1.0,NaN,False,False
1,31,1,0.0,NaN,False,0.0,53,0.0,0.0,False,False
2,83,1,1.0,1.0,True,0.0,52,1.0,NaN,False,True
3,75,0,3.0,NaN,False,0.0,55,0.0,1.0,False,False
5,62,0,0.0,4.0,False,1.0,50,0.0,NaN,False,False
6,85,1,1.0,1.0,False,0.0,53,0.0,NaN,False,False
7,39,0,2.0,3.0,True,1.0,50,1.0,1.0,False,True
9,98,0,0.0,1.0,False,NaN,48,0.0,0.0,True,False
10,64,0,0.0,0.0,True,0.0,54,0.0,NaN,False,True
12,109,1,1.0,0.0,False,0.0,48,1.0,1.0,True,False


In [126]:
testing.fillna({'family_inv': 0, 'mother_hs':1}).head(10)

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
0,40,0,2.0,2.0,False,NaN,55,1.0,1.0,False,False
1,31,1,0.0,0.0,False,0.0,53,0.0,0.0,False,False
2,83,1,1.0,1.0,True,0.0,52,1.0,1.0,False,True
3,75,0,3.0,0.0,False,0.0,55,0.0,1.0,False,False
5,62,0,0.0,4.0,False,1.0,50,0.0,1.0,False,False
6,85,1,1.0,1.0,False,0.0,53,0.0,1.0,False,False
7,39,0,2.0,3.0,True,1.0,50,1.0,1.0,False,True
9,98,0,0.0,1.0,False,NaN,48,0.0,0.0,True,False
10,64,0,0.0,0.0,True,0.0,54,0.0,1.0,False,True
12,109,1,1.0,0.0,False,0.0,48,1.0,1.0,True,False


Notice that `fillna` by default returns a new object with the desired filling behavior, rather than changing the `Series` or  `DataFrame` in place (**in general, we like to do this, by the way!**).

Missing values can also be interpolated, using any one of a variety of methods:

In [128]:
testing.fillna(method='bfill').head(10)

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
0,40,0,2.0,2.0,False,0.0,55,1.0,0.0,False,False
1,31,1,0.0,1.0,False,0.0,53,0.0,0.0,False,False
2,83,1,1.0,1.0,True,0.0,52,1.0,1.0,False,True
3,75,0,3.0,4.0,False,0.0,55,0.0,1.0,False,False
5,62,0,0.0,4.0,False,1.0,50,0.0,1.0,False,False
6,85,1,1.0,1.0,False,0.0,53,0.0,1.0,False,False
7,39,0,2.0,3.0,True,1.0,50,1.0,1.0,False,True
9,98,0,0.0,1.0,False,0.0,48,0.0,0.0,True,False
10,64,0,0.0,0.0,True,0.0,54,0.0,1.0,False,True
12,109,1,1.0,0.0,False,0.0,48,1.0,1.0,True,False


## Data summarization

We often wish to summarize data in `Series` or `DataFrame` objects, so that they can more easily be understood or compared with similar data. The NumPy package contains several functions that are useful here, but several summarization or reduction methods are built into Pandas data structures.

In [129]:
testing.sum()

score            17428.0
male               106.0
siblings           233.0
family_inv         170.0
non_english         42.0
prev_disab          45.0
age_test         11084.0
non_severe_hl       86.0
mother_hs           94.0
early_ident         78.0
non_white          100.0
dtype: float64

Clearly, `sum` is more meaningful for some columns than others. For methods like `mean` for which application to string variables is not just meaningless, but impossible, these columns are automatically exculded:

In [130]:
testing.mean()

score            84.193237
male              0.512077
siblings          1.125604
family_inv        0.977011
non_english       0.202899
prev_disab        0.238095
age_test         53.545894
non_severe_hl     0.415459
mother_hs         0.701493
early_ident       0.376812
non_white         0.483092
dtype: float64

The important difference between NumPy's functions and Pandas' methods is that the latter have built-in support for handling missing data.

Sometimes we may not want to ignore missing values, and allow the `nan` to propagate.

In [131]:
testing.mean(skipna=False)

score            84.193237
male              0.512077
siblings          1.125604
family_inv             NaN
non_english       0.202899
prev_disab             NaN
age_test         53.545894
non_severe_hl     0.415459
mother_hs              NaN
early_ident       0.376812
non_white         0.483092
dtype: float64

Passing `axis=1` will summarize over rows instead of columns, which only makes sense in certain situations.

In [132]:
nonenglish_nonwhite_withHS = testing[['non_english', 'mother_hs', 'non_white']].prod(axis=1)

nonenglish_nonwhite_withHS

0      0.0
1      0.0
2      1.0
3      0.0
5      0.0
6      0.0
7      1.0
9      0.0
10     1.0
12     0.0
13     0.0
14     0.0
16     0.0
17     1.0
18     0.0
19     0.0
21     0.0
23     0.0
25     0.0
26     0.0
27     0.0
28     0.0
29     1.0
30     1.0
31     0.0
32     0.0
33     0.0
34     0.0
35     1.0
36     0.0
      ... 
194    1.0
195    0.0
196    0.0
198    0.0
199    0.0
200    0.0
201    0.0
202    0.0
203    0.0
204    0.0
205    0.0
206    0.0
207    0.0
208    0.0
209    0.0
210    0.0
211    0.0
212    0.0
213    0.0
214    0.0
215    1.0
216    0.0
217    0.0
218    1.0
219    0.0
220    0.0
221    0.0
222    0.0
223    0.0
224    0.0
Length: 207, dtype: float64

A useful summarization that gives a quick snapshot of multiple statistics for a `Series` or `DataFrame` is `describe`:

In [133]:
testing.describe()

,score,male,siblings,family_inv,prev_disab,age_test,non_severe_hl,mother_hs
count,207.000000,207.000000,207.000000,174.000000,189.000000,207.000000,207.000000,134.000000
mean,84.193237,0.512077,1.125604,0.977011,0.238095,53.545894,0.415459,0.701493
std,24.885590,0.501066,0.866682,1.002621,0.427049,3.631920,0.493996,0.459320
min,0.000000,0.000000,0.000000,0.000000,0.000000,48.000000,0.000000,0.000000
25%,68.000000,0.000000,1.000000,0.000000,0.000000,50.000000,0.000000,0.000000
50%,87.000000,1.000000,1.000000,1.000000,0.000000,54.000000,0.000000,1.000000
75%,98.000000,1.000000,2.000000,2.000000,0.000000,57.000000,1.000000,1.000000
max,144.000000,1.000000,3.000000,4.000000,1.000000,59.000000,1.000000,1.000000


`describe` can detect non-numeric data and sometimes yield useful information about it.

In [134]:
testing.non_english.describe()

count       207
unique        2
top       False
freq        165
Name: non_english, dtype: object

We can also calculate summary statistics *across* multiple columns, for example, correlation and covariance.

$$cov(x,y) = \sum_i (x_i - \bar{x})(y_i - \bar{y})$$

In [135]:
testing.score.cov(testing.age_test)

4.6561371417850941

$$corr(x,y) = \frac{cov(x,y)}{(n-1)s_x s_y} = \frac{\sum_i (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_i (x_i - \bar{x})^2 \sum_i (y_i - \bar{y})^2}}$$

In [136]:
testing.score.corr(testing.age_test)

0.051515927472515266

Try running `corr` on the entire `testing` DataFrame to see what is returned:

In [ ]:
# Write answer here

If we have a `DataFrame` with a hierarchical index (or indices), summary statistics can be applied with respect to any of the index levels:

In [137]:
segments_hier.head()

seg_length  avg_sog  min_sog  \
mmsi name              year transit segment                                 
1    Bil Holman Dredge 2010 1       1              17.4     34.3     33.9   
                                    2              76.0     34.4     33.8   
     Dredge Capt Frank 2009 1       1              13.5     18.6     10.4   
                            2       1               9.2     15.4     14.6   
     S.d. Gumel        2011 1       1              13.7      6.9      3.0   

                                             max_sog  pdgt10  \
mmsi name              year transit segment                    
1    Bil Holman Dredge 2010 1       1           35.0   100.0   
                                    2           34.9   100.0   
     Dredge Capt Frank 2009 1       1           20.6   100.0   
                            2       1           16.2   100.0   
     S.d. Gumel        2011 1       1           14.0    38.2   

                                                        st_time       end_time  
mmsi name              year transit segment                                     
1    Bil Holman Dredge 2010 1       1       2010-03-20 16:06:00  3/20/10 16:36  
                                    2       2010-03-20 18:05:00  3/20/10 18:48  
     Dredge Capt Frank 2009 1       1       2009-04-06 14:31:00   4/6/09 15:20  
                            2       1       2009-04-10 17:59:00  4/10/09 18:35  
     S.d. Gumel        2011 1       1       2011-05-04 11:28:00   5/4/11 13:30

In [138]:
segments_hier.sum(level='transit').seg_length

transit
1       295064.7
2       275769.0
3       219727.9
4       205871.6
5       176446.5
6       163832.9
7       145289.1
8       138813.5
9       124391.2
10      117341.8
11      111900.5
12      109438.9
13       97329.0
14       88863.0
15       88306.5
16       88149.5
17       81462.1
18       76814.4
19       71675.8
20       69920.1
21       75222.1
22       66022.5
23       64807.7
24       64622.8
25       59666.6
26       58353.7
27       59193.7
28       51684.9
29       52314.1
30       50725.8
          ...   
2677         1.0
2686         2.0
2687         1.6
2694         1.0
2701         1.1
2702         3.3
2707         1.1
2708         1.3
2715         1.1
2720         1.4
2722         1.1
2723         1.3
2726         1.1
2729         1.2
2730         1.6
2731         1.5
2734         3.6
2735         1.6
2739         1.1
2744         1.7
2756         1.8
2757         1.7
2760         1.5
2768         1.9
2774         2.1
2796         1.3
2802         1.3
2818  

In [ ]:
pd.read_

## Writing Data to Files

As well as being able to read several data input formats, Pandas can also export data to a variety of storage formats. We will bring your attention to just a couple of these.

In [139]:
testing.to_csv("testing.csv")

In [140]:
!cat testing.csv

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
0,40,0,2.0,2.0,False,,55,1.0,,False,False
1,31,1,0.0,,False,0.0,53,0.0,0.0,False,False
2,83,1,1.0,1.0,True,0.0,52,1.0,,False,True
3,75,0,3.0,,False,0.0,55,0.0,1.0,False,False
5,62,0,0.0,4.0,False,1.0,50,0.0,,False,False
6,85,1,1.0,1.0,False,0.0,53,0.0,,False,False
7,39,0,2.0,3.0,True,1.0,50,1.0,1.0,False,True
9,98,0,0.0,1.0,False,,48,0.0,0.0,True,False
10,64,0,0.0,0.0,True,0.0,54,0.0,,False,True
12,109,1,1.0,0.0,False,0.0,48,1.0,1.0,True,False
13,111,0,0.0,0.0,False,1.0,48,1.0,,False,True
14,91,0,2.0,0.0,False,0.0,58,1.0,1.0,True,True
16,68,1,2.0,3.0,False,0.0,59,0.0,,True,True
17,87,0,1.0,0.0,True,0.0,58,0.0,,False,True
18,73,0,1.0,,False,0.0,54,1.0,1.0,False,True
19,65,1,0.0,2.0,False,0.0,50,1.0,1.0,False,True
21,71,1,1.0,2.0,False,1.0,58,0.0,0.0,True,True
23,82,0,1.0,0.0,False,1.0,59,0.0,1.0,True,False
25,97,0,1.0,0.0,False,1.0,56,1.0,1.0,False,False
26,95,1,1.0,0.0,True,0.0

The `to_csv` method writes a `DataFrame` to a comma-separated values (csv) file. You can specify custom delimiters (via `sep` argument), how missing values are written (via `na_rep` argument), whether the index is writen (via `index` argument), whether the header is included (via `header` argument), among other options.

An efficient way of storing data to disk is in binary format. Pandas supports this using Python’s built-in pickle serialization.

In [141]:
testing.to_pickle("testing_pickle")

The complement to `to_pickle` is the `read_pickle` function, which restores the pickle to a `DataFrame` or `Series`:

In [142]:
pd.read_pickle("testing_pickle").head()

,score,male,siblings,family_inv,non_english,prev_disab,age_test,non_severe_hl,mother_hs,early_ident,non_white
0,40,0,2.0,2.0,False,NaN,55,1.0,NaN,False,False
1,31,1,0.0,NaN,False,0.0,53,0.0,0.0,False,False
2,83,1,1.0,1.0,True,0.0,52,1.0,NaN,False,True
3,75,0,3.0,NaN,False,0.0,55,0.0,1.0,False,False
5,62,0,0.0,4.0,False,1.0,50,0.0,NaN,False,False


It is recommended that binary storage of data via pickle only be used as a temporary storage format, in situations where speed is relevant. This is because there is no guarantee that the pickle format will not change with future versions of Python.